## ipprl_tools Tutorial Notebook
This notebook is a walk-through of the following topics:
    1. Reading data using Pandas.
    2. Using Synthetic Data Generation Methods.
    3. Calculating Linkability Metrics on Generated Data.
    4. Writing data and metric information to file.

In [1]:
import pandas as pd
import numpy as np
from ipprl_tools import synthetic,metrics
from ipprl_tools.utils import tutorial,data

## 1. Reading Data Using Pandas

The module comes with a link to some pre-made synthetic data to demonstrate the corruption methods. To download it, we can use the `get_data()`  method from the `utils` package. 

In [2]:
path = tutorial.get_data()

This gets us the path to the data that has been pre-downloaded.
To read in the data, we use the `read_pickle()` method from `pandas`. We use this method because it can handle reading compressed ZIP files. If your data is in CSV format, you can also use `pandas.read_csv()` to read your data in.

In either case, the `data` variable will contain a Pandas DataFrame object after calling.

**Important Note:** In order for the corruption methods to work correctly, the DataFrame you use must be entirely of type `np.str`. The corruption methods expect to operate on strings, and many will break on non-string data. One easy way to make sure your DataFrame is of type `np.str` is to call the function `.astype(np.str)` when reading your data. This will cast all columns of the DataFrame to be of the correct type.

We can also print out a sample of the data using `<DataFrame>.head(<num_rows>)`

In [3]:
raw_data = pd.read_pickle(path).astype(np.str)
# Drop some of the unecessary columns in our dataset.
raw_data = raw_data.drop(["first_name","first_name2","last_name","last_name2","email","address","city2","zip2","state2"],axis=1)
# Rename the columns of our dataset.
raw_data.columns = ["first_name","last_name","email","address","ssn","sex","city","zip","state","dob","phone","phone2","phone3","race","pcp_npi","suffix","title"]
# Split the data into a dataset, and a swap set. We do this so that we can utilize the swap set in section 4.1.1
dataset = raw_data.iloc[:400000]
swap_set = raw_data.iloc[400000:]

dataset.head(5)

,first_name,last_name,email,address,ssn,sex,city,zip,state,dob,phone,phone2,phone3,race,pcp_npi,suffix,title
0,Isabelita,Dommersen,idommersen0@webs.com,48 Grover Way,105-17-1874,F,Houston,77281,Texas,2017/10/24,713-816-8206,651-608-1749,561-717-5270,Sri Lankan,76-5006664,Jr,Honorable
1,Byrom,Le Moucheux,blemoucheux1@cornell.edu,158 Marquette Hill,188-46-4510,M,Gainesville,30506,Georgia,2017/09/10,404-582-9658,502-478-1240,540-141-9416,Colville,49-7957492,Sr,Honorable
2,Garwin,Ismirnioglou,gismirnioglou2@army.mil,9538 Lighthouse Bay Circle,845-48-4845,M,South Bend,46620,Indiana,2017/07/22,574-885-2620,626-605-9078,406-221-1811,Asian Indian,68-4856593,Jr,Honorable
3,Ewan,Paquet,epaquet3@baidu.com,0123 Dawn Park,886-78-7800,M,Cumming,30130,Georgia,2017/05/26,706-761-4259,212-881-3527,502-205-2203,Honduran,78-9072361,Jr,Mr
4,Kamila,Tailour,ktailour4@rediff.com,8 Linden Terrace,617-90-0336,F,Pensacola,32595,Florida,2017/09/22,850-315-6220,605-784-3270,704-410-3803,Eskimo,95-6884148,II,Mr


## 2. Using Synthetic Data Generation Methods

Once the data is read in, we want to apply some corruption methods on it.

In [4]:
# We make a copy of the first few rows of data here so that we can compare it to the non-corrupted version.
data_to_corrupt = dataset.iloc[:5].copy()
# The indicators dictionary will hold some information about the corruptions as they are performed.
indicators = {}

In this example, we call the `drop_per_column()` method on our small amount of sample data. We pass the function:
1. `data` - The DataFrame holding our data.
2. `indicators` - A dictionary to hold some metadata about the corruptions.
3. `columns` - We pass `columns = None` to signify that we want this operation to run on *all* columns in the DataFrame.
4. `drop_pct` - This parameter tells the function what percentage of the rows should be dropped. In our case, we want to drop 50%.

In [5]:
synthetic.drop_per_column(data=data_to_corrupt,indicators=indicators,columns=None,drop_pct=0.5)

If we compare the original results to our corrupted version, we can see the the function has randomly deleted some elements of each row (The function rounded down from 50% to 2 rows).

In [6]:
comparison = data_to_corrupt.join(dataset.iloc[:5],lsuffix="_corrupt")
comparison[["first_name","first_name_corrupt","last_name","last_name_corrupt","address","address_corrupt"]]

,first_name,first_name_corrupt,last_name,last_name_corrupt,address,address_corrupt
0,Isabelita,Isabelita,Dommersen,Dommersen,48 Grover Way,
1,Byrom,,Le Moucheux,,158 Marquette Hill,158 Marquette Hill
2,Garwin,Garwin,Ismirnioglou,Ismirnioglou,9538 Lighthouse Bay Circle,
3,Ewan,,Paquet,Paquet,0123 Dawn Park,0123 Dawn Park
4,Kamila,Kamila,Tailour,,8 Linden Terrace,8 Linden Terrace


The indicators dictionary also contains information about which elements specifically were removed.

In [7]:
def get_metrics_row(metadata, row,num_columns):
    return [None if metadata.get((i,row)) is None else metadata.get((i,row)).keys() for i in range(num_columns)]

def make_df_from_metadata(metadata,data):
    num_columns = len(data.columns)
    
    metrics_df = pd.DataFrame.from_dict({idx : get_metrics_row(metadata,idx,num_columns) for idx in range(len(data))},orient="index",columns=data.columns)
    metrics_df["type"] = "metadata"
    
    tmp_data = data.copy()
    tmp_data["type"] = "data"
    
    
    visual_df = pd.concat([tmp_data,metrics_df]).set_index("type",append=True).sort_index()
    return visual_df

If we use the above helper functions above, we can view the corrupted data and the indicator metadata side-by-side. The indicator metadata records the corruptions, and in the case of more complex corruption methods, information about the corruption that was performed on each element of the synthetic dataset.

In [8]:
meta_df = make_df_from_metadata(indicators,data_to_corrupt)
meta_df

first_name          last_name                    email  \
  type                                                                      
0 data              Isabelita          Dommersen     idommersen0@webs.com   
  metadata               None               None                     None   
1 data                                                                      
  metadata  (drop_per_column)  (drop_per_column)        (drop_per_column)   
2 data                 Garwin       Ismirnioglou  gismirnioglou2@army.mil   
  metadata               None               None                     None   
3 data                                    Paquet                            
  metadata  (drop_per_column)               None        (drop_per_column)   
4 data                 Kamila                        ktailour4@rediff.com   
  metadata               None  (drop_per_column)                     None   

                       address                ssn                sex  \
  type                                                                 
0 data                                105-17-1874                  F   
  metadata   (drop_per_column)               None               None   
1 data      158 Marquette Hill        188-46-4510                  M   
  metadata                None               None               None   
2 data                                845-48-4845                  M   
  metadata   (drop_per_column)               None               None   
3 data          0123 Dawn Park                                         
  metadata                None  (drop_per_column)  (drop_per_column)   
4 data        8 Linden Terrace                                         
  metadata                None  (drop_per_column)  (drop_per_column)   

                         city                zip              state  \
  type                                                                
0 data                Houston              77281              Texas   
  metadata               None               None               None   
1 data                                                                
  metadata  (drop_per_column)  (drop_per_column)  (drop_per_column)   
2 data                                     46620            Indiana   
  metadata  (drop_per_column)               None               None   
3 data                Cumming              30130            Georgia   
  metadata               None               None               None   
4 data              Pensacola                                         
  metadata               None  (drop_per_column)  (drop_per_column)   

                          dob              phone             phone2  \
  type                                                                
0 data                                                 651-608-1749   
  metadata  (drop_per_column)  (drop_per_column)               None   
1 data             2017/09/10       404-582-9658                      
  metadata               None               None  (drop_per_column)   
2 data             2017/07/22       574-885-2620                      
  metadata               None               None  (drop_per_column)   
3 data             2017/05/26       706-761-4259       212-881-3527   
  metadata               None               None               None   
4 data                                                 605-784-3270   
  metadata  (drop_per_column)  (drop_per_column)               None   

                       phone3               race            pcp_npi  \
  type                                                                
0 data           561-717-5270                            76-5006664   
  metadata               None  (drop_per_column)               None   
1 data                                                   49-7957492   
  metadata  (drop_per_column)  (drop_per_column)               None   
2 data           406-221-1811       Asian Indian         68-4856593   
  metadata               N

## 2.1 Chaining Synthetic Methods

To generate a synthetic dataset suitable for linkage, we can call multiple synthetic data methods, one after another, on the same data. The end result of this chain is a dataset where multiple corruptions have been performed.

In [9]:
data_to_corrupt_large = dataset.iloc[:50].copy()
indicators_large = {}

In the below code, we chain together multiples calls to synthetic methods, passing the same data and indicator variables to each method. After calling the methods, we can print out the metadata DataFrame to see which corruptions were performed for each variable value.

In [10]:
insrt_columns = ["first_name","last_name","email"]
insrt_freqs = [0.2,0.2,0.5]
insrt_nums = [2,2,4]
synthetic.string_insert_alpha(data=data_to_corrupt_large,
                              indicators=indicators_large,
                              insrt_num=insrt_nums,
                              insrt_freq=insrt_freqs,
                              columns=insrt_columns)

n_insrt_columns = ["phone","ssn"]
n_insrt_freqs = [0.1,0.2]
n_insrt_nums = [2,2]
synthetic.string_insert_numeric(data=data_to_corrupt_large,
                                indicators=indicators_large,
                                insrt_num=n_insrt_nums,
                                insrt_freq=n_insrt_freqs,
                                columns=n_insrt_columns)

drop_cols = ["first_name","last_name","email","phone","ssn"]
drop_freqs = [0.2,0.1,0.5,0.4,0.1]
synthetic.drop_per_column(data=data_to_corrupt_large,
                          indicators=indicators_large,
                          columns=drop_cols,
                          drop_pct=drop_freqs)

In [11]:
large_meta_df = make_df_from_metadata(indicators_large,data_to_corrupt_large)
large_meta_df.head(10)

first_name              last_name  \
  type                                         
0 data      Isabelita                          
  metadata       None      (drop_per_column)   
1 data          Byrom            Le Moucheux   
  metadata       None                   None   
2 data         Garwin         Ismirnioglnoua   
  metadata       None  (string_insert_alpha)   
3 data           Ewan                 Paquet   
  metadata       None                   None   
4 data         Kamila                Tailour   
  metadata       None                   None   

                                             email  \
  type                                               
0 data                                               
  metadata                       (drop_per_column)   
1 data                bklemloucheux1@conrnell.evdu   
  metadata                   (string_insert_alpha)   
2 data                  gismirnioglou2@aermby.minl   
  metadata                   (string_insert_alpha)   
3 data                                               
  metadata                       (drop_per_column)   
4 data                                               
  metadata  (string_insert_alpha, drop_per_column)   

                               address                ssn   sex         city  \
  type                                                                         
0 data                   48 Grover Way        105-17-1874     F      Houston   
  metadata                        None               None  None         None   
1 data              158 Marquette Hill        188-46-4510     M  Gainesville   
  metadata                        None               None  None         None   
2 data      9538 Lighthouse Bay Circle        845-48-4845     M   South Bend   
  metadata                        None               None  None         None   
3 data                  0123 Dawn Park        886-78-7800     M      Cumming   
  metadata                        None               None  None         None   
4 data                8 Linden Terrace                        F    Pensacola   
  metadata                        None  (drop_per_column)  None         None   

              zip    state         dob              phone        phone2  \
  type                                                                    
0 data      77281    Texas  2017/10/24                     651-608-1749   
  metadata   None     None        None  (drop_per_column)          None   
1 data      30506  Georgia  2017/09/10       404-582-9658  502-478-1240   
  metadata   None     None        None               None          None   
2 data      46620  Indiana  2017/07/22       574-885-2620  626-605-9078   
  metadata   None     None        None               None          None   
3 data      30130  Georgia  2017/05/26       706-761-4259  212-881-3527   
  metadata   None     None        None               None          None   
4 data      32595  Florida  2017/09/22                     605-784-3270   
  metadata   None     None        None  (drop_per_column)          None   

                  phone3          race     pcp_npi suffix      title  
  type                                                                
0 data      561-717-5270    Sri Lankan  76-5006664     Jr  Honorable  
  metadata          None          None        None   None       None  
1 data      540-141-9416      Colville  49-7957492     Sr  Honorable  
  metadata          None          None        None   None       None  
2 data      406-221-1811  Asian Indian  68-4856593     Jr  Honorable  
  metadata          None          None        None   None       None  
3 data      502-205-2203      Honduran  78-9072361     Jr         Mr  
  metadata          None          None        None   None       None  
4 data      704-410-3803        Eskimo  95-6884148     II         Mr  
  metadata          None          None        None   None       None

We can save this information by writing it to an Excel file using the following command.

In [12]:
large_meta_df.to_excel("test_excel.xlsx")

## 3.1 Calculating Linkability Metrics

Once we have a dataset that has been sufficiently corrupted, we may want to calculate linkability measures on the data, to determine which columns we should use for linkage.

We can calculate metrics on the data using the `metrics` submodule.

In [13]:
metrics.run_metrics(data_to_corrupt_large)

,mdr,dvr,mean_gs,std_gs,max_gs,min_gs,entropy,ptme,atf
first_name,0.2,0.82,1.000000,0.000000,1,1,4.979471,92.943019,1.250000
last_name,0.1,0.92,1.000000,0.000000,1,1,5.411663,97.974159,1.111111
email,0.5,0.52,1.000000,0.000000,1,1,3.321928,70.672709,2.000000
address,0.0,1.00,1.000000,0.000000,1,1,5.643856,100.000000,1.000000
ssn,0.1,0.92,1.000000,0.000000,1,1,5.411663,97.974159,1.111111
sex,0.0,0.04,25.000000,4.000000,29,21,0.981454,98.145390,25.000000
city,0.0,0.90,1.111111,0.433191,3,1,5.413661,98.576211,1.111111
zip,0.0,1.00,1.000000,0.000000,1,1,5.643856,100.000000,1.000000
state,0.0,0.44,2.272727,1.710444,7,1,4.112949,92.230351,2.272727
dob,0.0,0.88,1.136364,0.343174,2,1,5.403856,98.982029,1.136364


Each row in the above DataFrame represents a column from the original dataset. The columns in the DataFrame are various Linkability Measures, which are calculated directly from the data. For more information about what these linkability measures mean, visit [this page.](https://github.com/cu-recordlinkage/iPPRL/blob/master/linkability/Metrics_Table.md)

## 4.1 Preparing Files for Linkage

To generate a dataset that we can use for linkage testing, we can use another function from the `utils.data` submodule.

In this example, we are now operating on `data`, which is the complete tutorial dataset we read in at the start of the notebook.

In [14]:
left_ds,right_ds,gt_labels = data.split_dataset(dataset,overlap_pct=0.2)

In the above line of code, we used the `split_dataset` function from `ipprl_tools.utils.data` to split the dataset for us. This function accepts a set of data and splits it into two datasets, each of which has some unique rows, and some rows that overlap with the other dataset. The exact amount of overlap is configurable with the `overlap_pct` parameter.

In this case, we chose to have 20% of the rows from `dataset` appear in both `left_ds` and `right_ds`. 

In addition to returning the two dataset variables, the function also returns a set of ground truth labels, `gt_labels`, which provide the `ID`s of the overlapping rows in `left_ds` and `right_ds`. If desired, you can evalaute the performance of your linkage using these known ground-truth labels.

### 4.1.1 Applying Corruption Methods
Like in Section 3, we will now apply corruption methods to the synthetic data. 

This time, we must operate on two datasets, `left_ds` and `right_ds`.

**Note:** These methods might take a long time to run, because they are operating on very large data. If you'd like them to finish quicker, you can pass a subset of the data (using the `.iloc` function of DataFrame) to the `split_dataset()` function above to make these operations complete quicker.

In [15]:
left_meta = {}
synthetic.string_transpose(left_ds,left_meta,4,0.05)
print("Transpose Complete.")
synthetic.string_delete(left_ds,left_meta,3,0.05)
print("Delete Complete.")
synthetic.string_insert_alpha(left_ds,left_meta,3,0.05,columns=["first_name","last_name","email","address","city","title"])
print("Insert Alpha Complete.")
synthetic.string_insert_numeric(left_ds,left_meta,3,0.05,columns=["phone","phone2","phone3","zip"])
print("Insert Numeric Complete.")
synthetic.edit_values(left_ds,swap_set,left_meta,0.1)
print("Edit Values Complete.")

columns = ["first_name",
            "last_name",
            "email",
            "address",
            "ssn",
            "sex",
            "city",
            "zip",
            "state",
            "dob",
            "phone",
            "phone2",
            "phone3",
            "race",
            "pcp_npi",
            "suffix",
            "title"]

drop_pcts = [0.03,
             0.03,
             0.75,
             0.06,
             0.25,
             0.07,
             0.07,
             0.07,
             0.02,
             0.02,
             0.85,
             0.85,
             0.2,
             0.2,
             0.99,
             0.2]

synthetic.drop_per_column(left_ds,left_meta,drop_pct=drop_pcts,columns=columns)
print("Per-Column Drop Complete.")

Transpose Complete.
Delete Complete.
Insert Alpha Complete.
Insert Numeric Complete.
Edit Values Complete.
Per-Column Drop Complete.


In [16]:
right_meta = {}
synthetic.string_transpose(right_ds,right_meta,4,0.05)
print("Transpose Complete.")
synthetic.string_delete(right_ds,right_meta,3,0.05)
print("Delete Complete.")
synthetic.string_insert_alpha(right_ds,right_meta,3,0.05,columns=["first_name","last_name","email","address","city","title"])
print("Insert Alpha Complete.")
synthetic.string_insert_numeric(right_ds,right_meta,3,0.05,columns=["phone","phone2","phone3","zip"])
print("Insert Numeric Complete.")
synthetic.edit_values(right_ds,swap_set,right_meta,0.1)
print("Edit Values Complete.")

columns = ["first_name",
            "last_name",
            "email",
            "address",
            "ssn",
            "sex",
            "city",
            "zip",
            "state",
            "dob",
            "phone",
            "phone2",
            "phone3",
            "race",
            "pcp_npi",
            "suffix",
            "title"]

r_drop_pcts = [0.05,
             0.03,
             0.75,
             0.06,
             0.25,
             0.07,
             0.07,
             0.07,
             0.02,
             0.02,
             0.80,
             0.80,
             0.2,
             0.2,
             0.99,
             0.2]

synthetic.drop_per_column(right_ds,right_meta,drop_pct=r_drop_pcts,columns=columns)
print("Per-Column Drop Complete.")

Transpose Complete.
Delete Complete.
Insert Alpha Complete.
Insert Numeric Complete.
Edit Values Complete.
Per-Column Drop Complete.


To verify that the corruption ran on both datasets, we can run the linkability metrics on both.

In [17]:
metrics.run_metrics(left_ds)

,mdr,dvr,mean_gs,std_gs,max_gs,min_gs,entropy,ptme,atf
first_name,0.03,0.121121,8.008807,11.659186,70,1,13.341810,89.982027,8.256502
last_name,0.03,0.290033,3.344491,2.410519,31,1,15.450870,96.045898,3.447929
email,0.75,0.249242,1.003059,0.056424,3,1,4.777909,30.109789,4.012237
address,0.06,0.898183,1.046562,0.237388,6,1,16.956304,95.702316,1.113364
ssn,0.25,0.743487,1.008765,0.099210,4,1,13.891159,79.628058,1.345020
sex,0.07,0.000013,111600.000000,366.000000,111966,111234,1.295916,81.763224,120000.000000
city,0.07,0.065821,14.130160,126.591573,6386,1,8.732796,62.612525,15.193720
zip,0.07,0.070850,13.127095,33.471840,144,1,11.141872,79.281341,14.115156
state,0.02,0.007942,123.464567,1092.886732,24028,1,5.549127,50.926555,125.984252
dob,0.02,0.033317,29.418386,126.522559,683,1,9.023693,69.600075,30.018762


In [18]:
metrics.run_metrics(right_ds)

,mdr,dvr,mean_gs,std_gs,max_gs,min_gs,entropy,ptme,atf
first_name,0.05,0.118633,8.008149,11.521909,70,1,13.153472,88.891291,8.429630
last_name,0.03,0.289067,3.355676,2.410262,26,1,15.448238,96.058297,3.459459
email,0.75,0.249312,1.002774,0.053544,3,1,4.778054,30.109924,4.011097
address,0.06,0.898300,1.046426,0.237438,6,1,16.956501,95.702421,1.113219
ssn,0.25,0.743425,1.008850,0.099911,4,1,13.891011,79.627767,1.345133
sex,0.07,0.000013,111600.000000,146.000000,111746,111454,1.295923,81.763607,120000.000000
city,0.07,0.066067,14.077578,125.705973,6274,1,8.745172,62.677086,15.137181
zip,0.07,0.071267,13.050342,33.369686,139,1,11.146468,79.266327,14.032626
state,0.02,0.008004,122.500000,1089.306058,24233,1,5.553737,50.916015,125.000000
dob,0.02,0.032537,30.122951,128.012787,692,1,9.017977,69.739625,30.737705


We can also look at the first few rows of the data.

In [19]:
left_ds.head()

,first_name,last_name,email,address,ssn,sex,city,zip,state,dob,phone,phone2,phone3,race,pcp_npi,suffix,title
id,,,,,,,,,,,,,,,,,
0,Chaddy,Wooller,,39 Randy Hill,413-19-0709,M,Buffalo,14269,New York,2018/02/26,,,405-411-8832,Spaniard,,Sr,Dr
1,Adriano,Di Angelo,,3 Hagan Circle,776-75-9488,M,Miami,33190,Florida,2017/07/09,,208-828-1705,540-633-1716,Chilean,,,Honorable
2,Lyell,Martinuzzi,lmartinuzzijnh@adobe.com,44558 Cody Hill,,M,Albuquerque,87110,New Mexico,2017/04/18,,,504-497-1949,Dominican (Dominican Republic),,,Mr
3,Forster,Risbrough,,24573 Messerschmidt Drive,,M,Albuquerque,87180,New Mexico,2017/05/18,,,,Alaska Native,,Sr,Mr
4,Patrizius,Hegerty,,8 Buhler Park,737-25-5721,M,,89012,Florida,2017/11/25,,,626-372-7830,White,,III,Mrs


In [20]:
right_ds.head()

,first_name,last_name,email,address,ssn,sex,city,zip,state,dob,phone,phone2,phone3,race,pcp_npi,suffix,title
id,,,,,,,,,,,,,,,,,
240000,Jo-ann,Wooller,,39 Randyill,,M,Buffalo,14269,New York,2018/02/26,,,763-391-7496,Spaniard,,Sr,Dr
240001,Krisha,Di Angelo,,8 Corscot Street,776-75-9488,M,Miami,33190,Florida,2017/07/09,,,540-633-1716,Lumbee,,Jr,Honorable
240002,Lyell,Martinuzzi,,44558 Cody Hill,634-14-4821,M,Albuquerque,87110,New Mexico,2017/04/18,,,504-497-1949,Dominican (Dominican Republic),,III,Rev
240003,Forster,Ribsorugh,,0055 Mitchell Center,108-75-5942,M,Albuquerque,,,2017/05/18,50-576-98994,,402-383-2415,Bangladeshi,,Sr,Mr
240004,Patrizius,Hegerty,phegerty1g2v@google.de,8 Buhler Park,737-25-5721,M,Henderson,8910112,Texas,2017/11/25,,,626-372-7830,,,,Mrs


We can now combine these two datasets into a single dataset in order to use it as input for linkage.

In [21]:
full_ds = pd.concat([left_ds,right_ds])

In [22]:
full_ds

,first_name,last_name,email,address,ssn,sex,city,zip,state,dob,phone,phone2,phone3,race,pcp_npi,suffix,title
id,,,,,,,,,,,,,,,,,
0,Chaddy,Wooller,,39 Randy Hill,413-19-0709,M,Buffalo,14269,New York,2018/02/26,,,405-411-8832,Spaniard,,Sr,Dr
1,Adriano,Di Angelo,,3 Hagan Circle,776-75-9488,M,Miami,33190,Florida,2017/07/09,,208-828-1705,540-633-1716,Chilean,,,Honorable
2,Lyell,Martinuzzi,lmartinuzzijnh@adobe.com,44558 Cody Hill,,M,Albuquerque,87110,New Mexico,2017/04/18,,,504-497-1949,Dominican (Dominican Republic),,,Mr
3,Forster,Risbrough,,24573 Messerschmidt Drive,,M,Albuquerque,87180,New Mexico,2017/05/18,,,,Alaska Native,,Sr,Mr
4,Patrizius,Hegerty,,8 Buhler Park,737-25-5721,M,,89012,Florida,2017/11/25,,,626-372-7830,White,,III,Mrs
5,Gerhard,Van Halen,,3893 6th Point,,M,Fort Lauderdale,33310,Florida,2017/10/04,754-813-8556,,952-822-2360,Pima,,Sr,Mrs
6,Haily,Kydde,hquarringtondcv@macromedia.com,7894 Rowland Plaza,732-69-1003,M,Jefferson City,24040,Virginia,2017/10/09,,,646-826-7237,Pueblo,,,Mrs
7,Colly,Romanin,chazart1ogi@mh.com.au,50152 Sycamore Terrace,404-48-1735,M,Kansabs City,64193,Missouri,201703/19,,,303-756-1512,Crow,,,Mrs
8,Shepard,Ivakhin,sivakhin16nr@oakley.com,96062 Golf Point,137-99-4619,M,Tucson,85743,Pennsylvania,2017/10/08,,616-841-7225,904-603-8818,Houma,,Jr,Rev


The `concat()` function will concatenate the two DataFrames into a single DataFrame along the axis. In our case, the `split_data()` utility function arranged it so that the indices of our index column `id`, are unique. If you did not use `split_data()` you'll want to make sure that you have references to the original IDs of your data so that you can evaluate the performance later.

`full_ds` is now a DataFrame which contains `left_ds` and `right_ds` stacked on top of each other (concatenated along the row dimension)

In [23]:
full_ds

,first_name,last_name,email,address,ssn,sex,city,zip,state,dob,phone,phone2,phone3,race,pcp_npi,suffix,title
id,,,,,,,,,,,,,,,,,
0,Chaddy,Wooller,,39 Randy Hill,413-19-0709,M,Buffalo,14269,New York,2018/02/26,,,405-411-8832,Spaniard,,Sr,Dr
1,Adriano,Di Angelo,,3 Hagan Circle,776-75-9488,M,Miami,33190,Florida,2017/07/09,,208-828-1705,540-633-1716,Chilean,,,Honorable
2,Lyell,Martinuzzi,lmartinuzzijnh@adobe.com,44558 Cody Hill,,M,Albuquerque,87110,New Mexico,2017/04/18,,,504-497-1949,Dominican (Dominican Republic),,,Mr
3,Forster,Risbrough,,24573 Messerschmidt Drive,,M,Albuquerque,87180,New Mexico,2017/05/18,,,,Alaska Native,,Sr,Mr
4,Patrizius,Hegerty,,8 Buhler Park,737-25-5721,M,,89012,Florida,2017/11/25,,,626-372-7830,White,,III,Mrs
5,Gerhard,Van Halen,,3893 6th Point,,M,Fort Lauderdale,33310,Florida,2017/10/04,754-813-8556,,952-822-2360,Pima,,Sr,Mrs
6,Haily,Kydde,hquarringtondcv@macromedia.com,7894 Rowland Plaza,732-69-1003,M,Jefferson City,24040,Virginia,2017/10/09,,,646-826-7237,Pueblo,,,Mrs
7,Colly,Romanin,chazart1ogi@mh.com.au,50152 Sycamore Terrace,404-48-1735,M,Kansabs City,64193,Missouri,201703/19,,,303-756-1512,Crow,,,Mrs
8,Shepard,Ivakhin,sivakhin16nr@oakley.com,96062 Golf Point,137-99-4619,M,Tucson,85743,Pennsylvania,2017/10/08,,616-841-7225,904-603-8818,Houma,,Jr,Rev


We can verify that the ground truth IDs from `split_data()` are still valid.

In [24]:
pair_num = 1
full_ds.loc[[gt_labels[pair_num][0],gt_labels[pair_num][1]]]

,first_name,last_name,email,address,ssn,sex,city,zip,state,dob,phone,phone2,phone3,race,pcp_npi,suffix,title
id,,,,,,,,,,,,,,,,,
1,Adriano,Di Angelo,,3 Hagan Circle,776-75-9488,M,Miami,33190,Florida,2017/07/09,,208-828-1705,540-633-1716,Chilean,,,Honorable
240001,Krisha,Di Angelo,,8 Corscot Street,776-75-9488,M,Miami,33190,Florida,2017/07/09,,,540-633-1716,Lumbee,,Jr,Honorable


Now we simply call `.to_csv()` to save our new dataset.

In [25]:
full_ds.to_csv("test_dataset.csv")

In order to evaluate performance later, it is also a good idea to save the individual meta objects as well as the ground-truth labels.

In [26]:
import pickle
# We can save the metadata files as .pkl files, which are a common binary format for Python.
pickle.dump(left_meta,open("left_meta.pkl","wb"))
pickle.dump(right_meta,open("right_meta.pkl","wb"))
# We'll save the ground truth labels into a pikcle as well.
pickle.dump(gt_labels,open("gt_labels.pkl","wb"))

To open these files again later, we can use the `pickle.load()` function in the same way we just used `pickle.dump()`

In [27]:
test_read = pickle.load(open("gt_labels.pkl","rb"))